In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/home/rohan/Documents/JupyterProjects/TensorflowProjects/RNN/FrozenDessert/Frozen_Dessert_Production.csv', index_col = 'DATE')

In [ ]:
df.head()

In [ ]:
split_ind = int(len(df) * 0.9)

In [ ]:
train = df.iloc[:split_ind]
test = df.iloc[split_ind:]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
generator = TimeseriesGenerator(scaled_train, scaled_train, length = 12, batch_size = 1)

val_generator = TimeseriesGenerator(scaled_test, scaled_test, length = 12, batch_size = 1)

In [ ]:
model = Sequential()


model.add(LSTM(100, activation = 'relu', input_shape = (12,1) ))

model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')

In [ ]:
early_stop = EarlyStopping(patience = 2, monitor = 'val_loss')


model.fit_generator(generator, 
                    validation_data = val_generator,
                    epochs = 20, callbacks = [early_stop])

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
test_predictions = []
current_batch = scaled_train[-12:].reshape(1,12,1)

for i in range(len(scaled_test)):
    
    current_pred = model.predict(current_batch)[0]
    
    test_predictions.append(current_pred)
    
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)

In [ ]:
test_predictions

In [ ]:
mean_squared_error(test_predictions, test) ** 0.5